
# 📓 Sales Analysis – Notebook

This notebook walks you through step-by-step:
1. Loading the sales_orders.cvs data  `sales_orders.csv`
2. 

In [2]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

BASE = os.getcwd()
DATA_CSV = os.path.join(BASE, "sales_orders.csv")
print("Data path:", DATA_CSV)


Data path: /Users/cezary/Python/sales_analysis/sales_orders.csv


## 1) Loading data and preview

In [3]:

df = pd.read_csv(DATA_CSV, parse_dates=["data"])
display(df.head(10))
df.info()


,id,produkt,kategoria,cena,data,klient
0,10001,Butelka filtrująca,Dom,102.29,2024-11-14,cust_0106
1,10001,Plecak City,Moda,92.30,2024-11-14,cust_0106
2,10002,Smartfon X,Elektronika,4472.14,2024-12-07,cust_0131
3,10002,Długopis Gel,Papiernicze,48.81,2024-12-07,cust_0131
4,10002,Klawiatura MX,Akcesoria,402.86,2024-12-07,cust_0131
5,10003,Klawiatura MX,Akcesoria,279.30,2024-07-15,cust_0231
6,10003,Bluza Hoodie,Moda,51.49,2024-07-15,cust_0231
7,10004,Biurko Slim,Meble,648.75,2024-05-06,cust_0228
8,10004,Koszulka Basic,Moda,68.40,2024-05-06,cust_0228
9,10005,Butelka filtrująca,Dom,151.61,2024-06-26,cust_0015


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1242 entries, 0 to 1241
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         1242 non-null   int64         
 1   produkt    1242 non-null   object        
 2   kategoria  1242 non-null   object        
 3   cena       1242 non-null   float64       
 4   data       1242 non-null   datetime64[ns]
 5   klient     1242 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 58.3+ KB
